In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

cam = 2
model_version = "my_model_23.0.keras"

Im_Width = 128
Im_Height = 128

def process_frame(frame, target_size):
    """
    Processes a webcam frame for model prediction.
    
    Args:
        frame (np.array): Captured frame from the webcam.
        target_size (tuple): Target size for resizing the frame (width, height).

    Returns:
        np.array: Preprocessed image ready for model prediction.
    """
    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

def predict_frame(frame, model, target_size):
    """
    Predicts the class of a single frame using the model.

    Args:
        frame (np.array): Captured frame from the webcam.
        model: Loaded Keras model.
        target_size (tuple): Target size for resizing the frame.

    Returns:
        str: Predicted class label.
        float: Confidence score.
    """
    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    predictions = model.predict(frame_array)  # Outputs probabilities for each class
    predicted_class = np.argmax(predictions[0])  # Get the index of the class with the highest probability
    confidence = predictions[0][predicted_class]  # Confidence score for the predicted class

    # Map class indices to labels
    class_labels = {0: "Defective", 2: "Non-Defective", 1: "No Bottle"}  # Update this to match your classes
    predicted_label = class_labels[predicted_class]

    print(f"Raw Predictions: {predictions[0]}")
    print(f"Predicted Class: {predicted_label}, Confidence: {confidence:.2f}")

    return predicted_label, confidence


def run_webcam_with_model(model_path, target_size=(Im_Width, Im_Height)):
    """
    Opens the webcam, captures frames, and processes them using the model.

    Args:
        model_path (str): Path to the saved Keras model.
        target_size (tuple): Target size for resizing frames (width, height).
    """
    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Open the webcam
    cap = cv2.VideoCapture(cam)
    if not cap.isOpened():
        raise IOError("Cannot open the webcam")

    print("Webcam opened. Press 'ESC' to exit.")
    try:
        while True:
            # Capture frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Process and predict
            feedback, confidence = predict_frame(frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            #prob = abs(round(confidence,2)- 0.5)*200

            # Display feedback on the frame
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            #cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 60),
                        #cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show the frame in a window
            cv2.imshow("Webcam Feed", frame)

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for ESC
                print("Exiting...")
                break

    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the script
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    run_webcam_with_model(model_path, target_size=(Im_Width, Im_Height))


Loading model...
Model loaded successfully.
Webcam opened. Press 'ESC' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Raw Predictions: [9.9997532e-01 2.9037119e-07 2.4492876e-05]
Predicted Class: Defective, Confidence: 1.00
Prediction: Defective (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Predictions: [4.1001737e-02 2.0415507e-06 9.5899618e-01]
Predicted Class: Non-Defective, Confidence: 0.96
Prediction: Non-Defective (Confidence: 0.96)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Predictions: [1.7647609e-02 1.1791441e-06 9.8235130e-01]
Predicted Class: Non-Defective, Confidence: 0.98
Prediction: Non-Defective (Confidence: 0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Raw Predictions: [1.44491885e-02 1.02055606e-06 9.85549748e-01]
Predicted Class: Non-Defective, Confidence: 0.99
Prediction: Non-Defective (Confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Raw Predictions: [1.6119460e-02 1.2438906e-06 9.8387927e-01]
Predicted Class: Non-Defective, Confidence: 0.98
Predict